<a href="https://colab.research.google.com/github/yyf999999999/Shogimodoki/blob/main/%E8%AA%B2%E9%A1%8C05_31_WT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **将棋もどき**

## **概要**

これは将棋を再現したプログラムです。しかし、相手の駒を取った後それを使うことや、相手陣地へ駒を進めると成るなどはできません。(タイトルが将棋もどきなのはそのため)また、一人で遊ぶことはできず、必ず二人必要です。

## **操作方法**

将棋の基本的なルールはこのサイトをご覧ください。
https://shogi-joutatsu.com/archives/category/hajimete/rule

開始すると、上から順に駒の座標、自分の駒、すべての駒の三つの盤面が表示されます。駒の座標は、|x座標y座標|というように一つのますに連続で表示されます。

その後動かす駒の座標入力を求められるので、駒の座標と自分の駒から動かそうと思う駒を選び、座標を入力してください。入力は x座標,y座標 のようにしてください。(例: x座標:4,y座標:8の駒を選択する=>4,8と入力)座標があっていない場合や相手の駒を選択した場合は、再度入力を求められます。また、違った形式や全角で入力した場合はエラーになります。

そうすると、動かせる位置一覧が表示されるので、そこから動かそうと思うものを左から何番目か入力してください。位置は[横にどれくらい動かすか,縦にどれくらい動かすか]の順で表示されます。番号は一番左を1として数えてください。(例: [1,0],[-1,2],[0,1] これに2と入力=>左に1、上に2動かす)動かせる場所がない場合はそのまま相手のターンへ移行します。表示されている数を超えたり全角で入力するとエラーになります。

これらの動作が完了すると、駒の動きが反映されてターンが切り替わります。この際盤面の上下左右も逆になるので気を付けてください。これを繰り返し、相手の王を取ったほうが勝ちです。もし途中でエラーになった場合はもう一度メイン処理を実行すれば再開できますが、必ず一人目のターンになります。

## **制作者のコメント**

制作中はほとんどの時間をバグの解消に使いました。制作で最もつらかったことは、何十分もかけて直したバグが次の授業で初心者がやりがちなこととして一分程度で読めるテキストに載っていたことです。

## **作成時間**

作成時間：**約13時間**



In [1]:
#@title 初期化 ( %reset -f )
%reset -f

In [2]:
#@title サブ処理
# 各種ライブラリのインポート
# データやパラメータ設定
# 関数やクラスなどの定義
import IPython.display
import time

gameresult=[False,""]
out_numbers=[]
a=[]
player=["二人目","一人目"]
pattern=[[1,0],[0,-1],[-1,0],[0,1],[1,1],[-1,1],[1,-1],[-1,-1]]
stage = [["  "]*10 for i in range(10)]
piece=[None]*40
#必要なグローバル変数を定義する
piece_maker={"name":["王","金","銀","桂","香","角","飛","歩"],#駒の初期設定を作るための辞書を定義する
             "number":[1,2,2,2,2,1,1,9],#駒を一人当たり何個作るか決める
             "x":[[4],[3,5],[2,6],[1,7],[0,8],[1],[7],[i for i in range(9)]],#駒の初期x座標の基準を決める
             "y":[0,0,0,0,0,1,1,2],#駒の初期y座標の基準を決める
             "move":[pattern,pattern[:-2],pattern[3:],[[1,2],[-1,2]],#王,金,銀,桂馬の移動できる座標を入れる
                     [[0,i] for i in range(1,9)],#香車の移動できる座標を入れる
                     [[i*n,i*j] for i in range(1,9) for n in range(1,-2,-2) for j in range(1,-2,-2)],#角行の移動できる座標を入れる
                     [[i*n,0] for i in range(1,9) for n in range(1,-2,-2)]+[[0,i*n] for i in range(1,9) for n in range(1,-2,-2)],#飛車の移動できる座標を入れる
                     [[0,1]]]}#歩の移動できる座標を入れる
for j in range(1,-2,-2):#2回繰り返し、1,-1の順で数を与えることで一人目と二人目を区別する
  for i,num in enumerate(piece_maker["number"]):#駒を何個作るか書かれた配列をそれぞれnumに入れ、種類分繰り返す
    for n in range(num):#駒を作る数だけ繰り返す
      position=sum(piece_maker["number"][:i])+n-(j-1)*10#リストpieceの値が代入されていない最低値を求める
      piece[position]=({"name":piece_maker["name"][i],#求めた場所に駒の初期設定を入れる
                        "x":int(abs(4*(j+1)-piece_maker["x"][i][n])),#j(-1と1のどちらか)の値を使って一人目と二人目の駒のx座標を逆にする
                        "y":piece_maker["y"][i]*j,#j(-1と1のどちらか)の値を使って一人目と二人目の駒のy座標を逆にする
                        "direction":j,#j(-1と1のどちらか)を向きとする
                        "owner":player[int(j/2-0.5)],#j(-1と1のどちらか)を0と-1に変え、一人目か二人目かの名前を入れる
                        "move":piece_maker["move"][i]})#駒の種類に応じて移動できる座標を入れる
      stage[piece[position]["y"]][piece[position]["x"]]=piece[position]["name"]#駒を初期設定の位置に置く
  for i in range(len(piece_maker["y"])):#一人当たりの駒の数分繰り返す
    piece_maker["y"][i]+=2#一人目と二人目の駒のy座標がズレるのを直す
for i in piece:#すべての駒で処理を行う
  if i["y"]<0:
    i["y"]+=10#駒のy座標の負を正に変える
#my_piece=[i for i in piece if i["owner"]=="一人目"]#一人目の駒を集める
#enemy_piece=[i for i in piece if i["owner"]=="二人目"]#二人目の駒を集める
def printer(c,s,e):#print(c,s,e)と全く同じ動作をする関数なぜ作成したかは不明
  for i in c[:-1]:#描画する要素の長さ-1繰り返す
    print(i,s,sep="",end="")#通常の内容と間に入れる内容を描画する
  print(c[-1],e,sep="",end="")#通常の内容と終わりの内容を描画する
def set_display(sp,numbers,limit,po):#sp:一人目と二人目の上下左右を変える,numbers:描画するのが座標(True)か座標にあるもの(False)か,limit:自分のみ映す(True)かすべて映す(False)か,po:一人目か二人目か
  for i in range(9):#y座標分繰り返す
    for n in range(28):#マスを区切る横線を描画する
      printer(["-"],"","")
    printer(["","|"],"\n","")#改行し、左端の縦線を描画する
    for n in range(9):#x座標分繰り返す
      if numbers:#数字を描画するかどうか
        printer([int(abs(sp-n)),int(abs(sp-i)),"|"],"","")#x,yと描画する
      else:
        if limit and stage[int(abs(sp-i))][int(abs(sp-n))]!="  ":#自分の駒のみ出力し、かつ描画予定の座標に駒があるか
          if search_piece(int(abs(sp-n)),int(abs(sp-i)),False)["owner"]==po:#描画予定の座標の駒が自分のものかどうか
            printer([stage[int(abs(sp-i))][int(abs(sp-n))],"|"],"","")#駒を描画する
          else:
            printer(["  ","|"],"","")#描画しない(空白を描画)
        else:
          printer([stage[int(abs(sp-i))][int(abs(sp-n))],"|"],"","")#描画予定の座標の駒を描写する(空白も可)
    print()#改行する
  for n in range(28):#最後のマス区切り横線を描画する
    printer(["-"],"","")
  print()#改行する
def search_piece(x,y,change):#選択したx座標,選択したy座標,駒のリストのインデックスで返すかどうか
  for a in piece:#すべての駒で試す
    if a["x"]==x and a["y"]==y:#選んだ駒の座標が選択した座標と同じか
      if change:#インデックスで返すかどうか
        return(piece.index(a))#選択した座標の駒のインデックスを返す
      else:
        return(a)#選択した座標の駒の情報を返す
  return None#見つからなかった場合は、Noneを返す
def decide_direction(inum):#動かす駒(飛車)のx,y座標が+か0か-かを返す
  v=[None]*len(inum)#x,y座標を入れるリストを作る
  order=0#リストの何番目に入れるか決める変数を初期化する
  for n in inum:#x,y座標でこの処理を行う
    if n!=0:#+か-ならば
      n/=abs(n)#絶対値で割る
    v[order]=int(n)#得た値をリストに入れる
    order+=1#値を入れるリストの番号を変える
  return(v[0]*2+v[1])#+か-,0の組み合わせによって違う値を返す(例:[1,0]=>2,[0,1]=>1,[-1,0]=>-2(=3),[0,-1]=>-1(=4))
def action(pn,sp):#メインの処理
  while True:#breakするまで繰り返す
    way=[9,9]#駒の座標を何もないところにする
    while stage[way[1]][way[0]]=="  ":#駒の座標に何もない間
      way=input(f"{player[pn]}が動かす駒( x座標,y座標 で指定 例:4,8):").split(",")#駒の座標の入力を求め、入力されたものを分解してリストにする
      way=[int(i) for i in way]#入力されたstr型の値をint型に変える
    select_piece=search_piece(way[0],way[1],False)|{"realmove":[],"excutionmove":None}#入力された座標にある駒を取得する
    if select_piece["owner"]==player[pn]:#もし取得した駒が自分のものならば
      break#この処理を終わらせる
  out_numbers=[True,True,True,True]#角行が誰かの駒に当たった時の飛び越え制限
  out_directions=[True,True,True,True,True]#飛車が誰かの駒に当たった時の飛び越え制限
  for i in select_piece["move"]:#すべての動き方で判定する
    d=select_piece["direction"]#向きをdという変数に入れる
    direction=decide_direction(i)#駒が動く向きを取得する(実際に処理で使うのは飛車のみ)
    if 0<=way[0]+i[0]*-d<9 and 0<=way[1]+i[1]*d<9 and out_numbers[select_piece["move"].index(i)%4] and out_directions[direction]:#駒が場外に出ないかつすでに角行や飛車で誰かの駒にぶつかっていないならば
      if stage[way[1]+i[1]*d][way[0]+i[0]*-d]=="  ":#もし動く先に何もないならば
        select_piece["realmove"].append(i)#動ける方法として保存する
      else:
        if search_piece(way[0]+i[0]*-d,way[1]+i[1]*d,False)["owner"]==player[int(abs(pn-1))]:#もしぶつかり先の駒が相手のものならば
          select_piece["realmove"].append(i)#動ける先として追加する
        if select_piece["name"]=="香":#もし駒の種類が香車ならば
          break#処理を中断し、これ以上動き先を追加できないようにする
        elif select_piece["name"]=="角":#もし駒の種類が角行ならば
          out_numbers[select_piece["move"].index(i)%4]=False#この向きでこれより先に動けないようにする
        elif select_piece["name"]=="飛":#もし駒の種類が飛車ならば
          direction=decide_direction(i)#動きの向きを取得する
          out_directions[direction]=False#この向きでこれより先に動けないようにする
  if len(select_piece["realmove"])>0:#もし動き方があるのならば
    printer(["移動できる方向:",*select_piece["realmove"]]," ","\n")#動き方を描画する
    select_piece["excutionmove"]=select_piece["realmove"][int(input("左から何番目の通りに動かすか入力してください([x,y]の順,例:[1,0] [0,1] 2を入力->[0,1]の通りに動く)"))-1]#どの方法で動かすか決める
    stage[select_piece["y"]][select_piece["x"]]="  "#自分のもともといた座標を消す
    select_piece["x"]+=select_piece["excutionmove"][0]*-select_piece["direction"]#駒の座標を動かす
    select_piece["y"]+=select_piece["excutionmove"][1]*select_piece["direction"]#駒の座標を動かす
    if stage[select_piece["y"]][select_piece["x"]]!="  ":#もし動く先に駒があるのならば
      delete_piece=piece[search_piece(select_piece["x"],select_piece["y"],True)]#その駒を取得する
      piece.remove(delete_piece)#その駒を消す
      if delete_piece["name"]=="王":#もし消した駒の種類が王ならば
        gameresult[0]=True;gameresult[1]=player[pn]#ゲームがもう終わるようにし、勝者を決める
    sbj=piece[search_piece(select_piece["x"]-select_piece["excutionmove"][0]*-select_piece["direction"],select_piece["y"]-select_piece["excutionmove"][1]*select_piece["direction"],True)]#すべての駒を管理するリストにある動かした座標を取得する
    sbj["x"]+=select_piece["excutionmove"][0]*-select_piece["direction"]#すべての駒を管理するリストの動かした駒のx座標を変える
    sbj["y"]+=select_piece["excutionmove"][1]*select_piece["direction"]#すべての駒を管理するリストの動かした駒のy座標を変える
    stage[select_piece["y"]][select_piece["x"]]=select_piece["name"]#盤面の座標に動かした駒をいれる
  else:#もしうごける方法がないのならば
    printer(["移動できる方向はありません"],"","\n")#"移動できる方向はありません"と描画する
    time.sleep(3)#3秒待つ
  IPython.display.clear_output()#ディスプレイをきれいにする
  printer(["駒の座標(全て|xy|の順で刻まれている)"],"","\n")#駒の座標表の読み方を解説する
  set_display(sp,True,False,player[int(abs(pn-1))])#駒の座標表を描画する
  printer([f"{player[int(abs(pn-1))]}の駒"],"","\n")#自分の駒の描画するという
  set_display(sp,False,True,player[int(abs(pn-1))])#自分の駒を描画する
  printer(["実際の駒"],"","\n")#全部の駒を描画するという
  set_display(sp,False,False,player[int(abs(pn-1))])#全部の駒を描画する

In [3]:
#@title メイン処理
# プログラムのなかには適切にコメントを記述してください
printer(["駒の座標(全て|xy|の順で刻まれている)"],"","\n")#駒の座標表の読み方を解説する
set_display(0,True,False,"一人目")#駒の座標表を描画する
printer(["一人目の駒"],"","\n")#自分の駒の描画するという
set_display(0,False,True,"一人目")#自分の駒を描画する
printer(["実際の駒"],"","\n")#全部の駒を描画するという
set_display(0,False,False,"一人目")#全部の駒を描画する
while True:#試合が終わるまで繰り返す
  action(1,8)#一人目の処理をする
  if gameresult[0]:#もし試合が終了しているなら
    printer([f"{gameresult[1]}の勝ちです"],"","\n")#王を取ったほうの勝ちを宣言する
    break#プログラムを終了する
  action(0,0)#二人目の処理をする
  if gameresult[0]:#もし試合が終了しているなら
    printer([f"{gameresult[1]}の勝ちです"],"","\n")#王を取ったほうの勝ちを宣言する
    break#プログラムを終了する

駒の座標(全て|xy|の順で刻まれている)
----------------------------
|88|78|68|58|48|38|28|18|08|
----------------------------
|87|77|67|57|47|37|27|17|07|
----------------------------
|86|76|66|56|46|36|26|16|06|
----------------------------
|85|75|65|55|45|35|25|15|05|
----------------------------
|84|74|64|54|44|34|24|14|04|
----------------------------
|83|73|63|53|43|33|23|13|03|
----------------------------
|82|72|62|52|42|32|22|12|02|
----------------------------
|81|71|61|51|41|31|21|11|01|
----------------------------
|80|70|60|50|40|30|20|10|00|
----------------------------
二人目の駒
----------------------------
|  |  |  |  |  |  |  |  |  |
----------------------------
|  |  |  |  |  |  |  |  |  |
----------------------------
|  |  |  |  |  |  |  |  |  |
----------------------------
|  |  |  |  |  |  |  |  |  |
----------------------------
|  |  |  |  |  |  |  |  |  |
----------------------------
|  |  |  |  |歩|  |  |  |  |
----------------------------
|歩|歩|歩|歩|  |歩|歩|歩|歩|
------------------------